In [1]:
import os
import pandas as pd
from collections import Counter

import matplotlib.pyplot as plt

In [2]:
asset = 'AA'

In [26]:
df = pd.read_parquet('./prices/AA_all.parquet')
df = df[df.regularMarketTime > '2024-08-00']
pre_df = df[df.marketState == 'PRE'].groupby('regularMarketTime').last()
market_df = df[df.marketState == 'REGULAR'].set_index('regularMarketTime')

all_df = pd.concat([pre_df, market_df]).reset_index()
all_df['dt'] = all_df.regularMarketTime.str.split().str[0]
all_df.head()

,regularMarketTime,symbol,quoteType,marketState,regularMarketChangePercent,regularMarketPrice,regularMarketChange,regularMarketOpen,regularMarketDayHigh,regularMarketDayLow,...,trailingPE,priceEpsCurrentYear,fiftyDayAverageChange,fiftyDayAverageChangePercent,twoHundredDayAverageChange,twoHundredDayAverageChangePercent,marketCap,forwardPE,priceToBook,dt
0,2024-08-01 16:00:02Z,AA,EQUITY,PRE,-4.751820,31.47,-1.570000,32.95,33.270,31.090,...,NaN,55.210526,-7.600802,-0.194539,-1.022551,-0.031470,8129739264,12.150579,1.449295,2024-08-01
1,2024-08-02 16:00:02Z,AA,EQUITY,PRE,-6.355260,29.47,-2.000000,31.05,31.055,29.270,...,NaN,52.625000,-9.308401,-0.240041,-3.035852,-0.093394,7613072896,11.511719,1.952561,2024-08-02
2,2024-08-05 16:00:02Z,AA,EQUITY,PRE,-3.495080,28.44,-1.030000,27.98,28.540,27.130,...,NaN,50.785713,-10.064199,-0.261379,-4.069200,-0.125171,7347189760,11.109375,1.884317,2024-08-05
3,2024-08-06 16:00:02Z,AA,EQUITY,PRE,3.340360,29.39,0.949999,28.50,29.640,28.215,...,NaN,40.819443,-8.881001,-0.232056,-3.133549,-0.096347,7592612352,10.534050,1.947260,2024-08-06
4,2024-08-07 16:00:02Z,AA,EQUITY,PRE,-0.612454,29.21,-0.180000,30.00,30.515,28.930,...,NaN,40.569443,-8.812401,-0.231769,-3.338402,-0.102567,7546110976,10.469534,1.935334,2024-08-07


In [49]:
funds = 4000
qty = 100
commission = 1.25
cur_qty = 0
avg_price = 0
for dt, dt_df in all_df.groupby('dt'):
    print(dt)
    pre_series = dt_df[dt_df.marketState == 'PRE']
    if len(pre_series['regularMarketPrice']) == 0: continue
    pre_price = pre_series['regularMarketPrice'].iloc[0]
    
    init_buy, init_sell = pre_price, pre_price * 1.1
    print(pre_price, init_buy, init_sell)
    
    first_trade = True
    
    for ix, row in dt_df[dt_df.marketState == 'REGULAR'].sort_values(by=['regularMarketTime']).iterrows():
        cur_price = row['regularMarketPrice']
        if first_trade and row['regularMarketPrice'] <= init_buy:
            # First Buy
            cur_qty += qty
            trade_price = cur_price * qty + commission
            funds -= trade_price 
            avg_price = cur_price
            first_trade = False
            print(f'Bought {qty} @ {cur_price}: {trade_price}')
        elif row['regularMarketPrice'] <= avg_price - 0.10:
            # Subsequent Buys
            buyable = max(min(funds // (cur_price + 1), qty), 0)
            avg_price = ( avg_price * cur_qty + buyable * cur_price ) / (cur_qty + buyable)
            cur_qty += buyable
            funds -= cur_price * buyable + commission
            
            print(f'Bought {buyable} @ {cur_price}: {trade_price}')
            print(f'-- Average {cur_qty} @ {avg_price}')
        elif cur_qty > 0 and row['regularMarketPrice'] >= avg_price + 0.10:
            # Sells
            avg_price = (avg_price * cur_qty - qty * cur_price ) / (cur_qty - qty) if cur_qty > qty else 0
            cur_qty -= qty
            trade_price = cur_price * qty - commission
            funds += trade_price
            print(f'Sold {qty} @ {cur_price}: {trade_price}')
            print(f'-- Average {cur_qty} @ {avg_price}')
    print(f'Funds: {funds}')
    print('='*50)

2024-08-01
31.47 31.47 34.617000000000004
Bought 100 @ 31.4: 3141.25
Sold 100 @ 31.54: 3152.75
-- Average 0 @ 0
Funds: 4011.5
2024-08-02
29.47 29.47 32.417
Bought 100 @ 29.465: 2947.75
Sold 100 @ 29.565: 2955.25
-- Average 0 @ 0
Funds: 4019.0
2024-08-05
28.44 28.44 31.284000000000002
Bought 100 @ 27.99: 2800.25
Bought 42.0 @ 27.85: 2800.25
-- Average 142.0 @ 27.948591549295774
Bought 1.0 @ 27.82: 2800.25
-- Average 143.0 @ 27.947692307692307
Bought 0.0 @ 27.8198: 2800.25
-- Average 143.0 @ 27.947692307692307
Bought 0.0 @ 27.83: 2800.25
-- Average 143.0 @ 27.947692307692307
Bought 0.0 @ 27.735: 2800.25
-- Average 143.0 @ 27.947692307692307
Bought 0.0 @ 27.221: 2800.25
-- Average 143.0 @ 27.947692307692307
Bought 0.0 @ 27.39: 2800.25
-- Average 143.0 @ 27.947692307692307
Bought 0.0 @ 27.18: 2800.25
-- Average 143.0 @ 27.947692307692307
Bought 0.0 @ 27.46: 2800.25
-- Average 143.0 @ 27.947692307692307
Bought 0.0 @ 27.51: 2800.25
-- Average 143.0 @ 27.947692307692307
Bought 0.0 @ 27.48: 28